In [1]:
# !pip install -q timm
from transformers import DetrFeatureExtractor
import torch
import fitz  # PyMuPDF
from PIL import Image
from transformers import TableTransformerForObjectDetection
import matplotlib.pyplot as plt
from huggingface_hub import hf_hub_download
import camelot




[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
path to folder of documents


folder of saved CVS

In [114]:

def get_image(folder_path: str):
    files = {}

    # Iterate through each file in the folder
    for file in os.listdir(folder_path):
        images = []
        widths = []
        heights = []
        # Check if the file is a PDF
        if file.endswith(".pdf"):
            file_path = os.path.join(folder_path, file)

            # Open the PDF file
            pdf_document = fitz.open(file_path)
            print(file)
            # Process a specific page (e.g., page number 132)
            # page_number = 132
            for page_number in range(len(pdf_document)):
                # if page_number < len(pdf_document):
                page = pdf_document.load_page(page_number)
                pix = page.get_pixmap()
                image_path = f"page_{page_number}_{file}.png"
                pix.save(image_path)

                # Open and process the image with PIL
                image = Image.open(image_path).convert("RGB")
                width, height = image.size

                # Store the image and its dimensions
                images.append(image)
                widths.append(width)
                heights.append(height)

        files[file] = {"name": file, "images": images, "widths": widths, "heights": heights}

    return files

# Usage example
# files = get_images("/Users/danielbasman/Desktop/Desktop - Daniel’s MacBook Pro/MyStuff/Classes/Human In Loop/folder_of_10K")
# print(files)


In [29]:
# print(files["name"])
print((files["10-K.pdf"]))
print((files["10-K-EX2.pdf"]))

{'name': '10-K.pdf', 'images': [<PIL.Image.Image image mode=RGB size=612x792 at 0x15E06F460>, <PIL.Image.Image image mode=RGB size=612x792 at 0x15E06FD00>, <PIL.Image.Image image mode=RGB size=612x792 at 0x15E06FBE0>, <PIL.Image.Image image mode=RGB size=612x792 at 0x15E06E7A0>, <PIL.Image.Image image mode=RGB size=612x792 at 0x15E06FD90>, <PIL.Image.Image image mode=RGB size=612x792 at 0x15E068A30>, <PIL.Image.Image image mode=RGB size=612x792 at 0x15E0686A0>, <PIL.Image.Image image mode=RGB size=612x792 at 0x15E06FBB0>, <PIL.Image.Image image mode=RGB size=612x792 at 0x15E06F0D0>, <PIL.Image.Image image mode=RGB size=612x792 at 0x15E19D0F0>, <PIL.Image.Image image mode=RGB size=612x792 at 0x15E076DD0>, <PIL.Image.Image image mode=RGB size=612x792 at 0x15E085DB0>, <PIL.Image.Image image mode=RGB size=612x792 at 0x15E085EA0>, <PIL.Image.Image image mode=RGB size=612x792 at 0x15E085E10>, <PIL.Image.Image image mode=RGB size=612x792 at 0x15E085DE0>, <PIL.Image.Image image mode=RGB size=6

In [115]:
def extract_features(image, width, height, model):

  feature_extractor = DetrFeatureExtractor()
  encoding = feature_extractor(image, return_tensors="pt")
  # encoding.keys()
  # print(encoding['pixel_values'].shape)


  with torch.no_grad():
    outputs = model(**encoding)
    results = feature_extractor.post_process_object_detection(outputs, threshold=0.001, target_sizes=[(height, width)])[0]

    return results, feature_extractor
# model, results, feature_extractor = extract_features(img, width, height)

In [116]:

def plot_box(pil_img, boxes):
    COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]

    plt.figure(figsize=(16,10))
    plt.imshow(pil_img)
    ax = plt.gca()
    
    ax.add_patch(plt.Rectangle((boxes[1], boxes[2]), boxes[3] - boxes[1], -(boxes[2] - boxes[0]),
                                   fill=False, color='red', linewidth=3))
    
    print("height: ", boxes[2] - boxes[0])
    print("width: ", boxes[3] - boxes[1])
    plt.axis('on')

    plt.show()
    
def plot_results(pil_img, scores, labels, boxes):
    COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]
    plt.figure(figsize=(16,10))
    plt.imshow(pil_img)
    ax = plt.gca()
    colors = COLORS * 100
    for score, label, (xmin, ymin, xmax, ymax),c  in zip(scores.tolist(), labels.tolist(), boxes.tolist(), colors):
        ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                   fill=False, color=c, linewidth=3))
        text = f'{model.config.id2label[label]}: {score:0.2f}'
        ax.text(xmin, ymin, text, fontsize=8,
                bbox=dict(facecolor='yellow', alpha=0.5))
    plt.axis('on')

    plt.show()

In [117]:
def sort_results(results):

    # width, height = image.size

    max_score_index = torch.argmax(results['scores'])
    max_score = results['scores'][max_score_index]
    max_score_box = results['boxes'][max_score_index]

    left = []
    right = []
    top = []
    bottom = []

    for i in results["boxes"]:
        top.append(i[3])
        right.append(i[2])
        left.append(i[0])
        bottom.append(i[1])
        # print("top y: ", i[3], "bottom y: ", i[1], "right x: ", i[2], "left x: ", i[0])
        
    # print(left)
    final = []

    for i in [bottom, left]:
        # print(i)
        min_value = min(tensor.min() for tensor in i)
        # print(min_value)
        final.append(min_value)

    for i in [top, right]:
        # print(i)
        max_value = max(tensor.max() for tensor in i)
        # print(max_value)
        final.append(max_value)
    # print(final)
    # print("Anchor")
    # print(final[1])
    # print(final[2])
    max_score_box = [max_score_box[3], max_score_box[0], max_score_box[1], max_score_box[2]]
    # print(max_score_box)

    return final, max_score_box

# final, max_score_box = sort_results(results)


In [118]:

def convert_results(results):
    # Converting tensors to lists
    data = {key: value.tolist() for key, value in results.items()}

    # Combining the scores, labels, and boxes into a single list of tuples for sorting
    combined_data = list(zip(data['scores'], data['labels'], data['boxes']))

    # Sorting the combined data by scores in descending order
    sorted_data = sorted(combined_data, key=lambda x: x[0], reverse=True)

    # Reformatting back into the dictionary structure
    sorted_dict = {
        'scores': [item[0] for item in sorted_data],
        'labels': [item[1] for item in sorted_data],
        'boxes': [item[2] for item in sorted_data]
    }

    return sorted_dict

In [119]:
def get_table_image(image, final):
    width, height = image.size
    image.resize((int(width*0.5), int(height*0.5)))

    x =  int(final[1])
    y = int(final[0])
    width = int(final[3] - final[1])
    height = int(final[2] - final[0])
    print(x, y, width, height)
    cropped_image = image.crop((x-30, y-20, x + width + 30, y + height))
    # cropped_image.save('cropped_portion.png')

    return cropped_image

# cropped_image = get_table_image(image, final)


In [120]:
def table_structure_results(cropped_image, feature_extractor):

  image = cropped_image.convert("RGB")

  width, height = image.size
  # image.resize((int(width*0.5), int(height*0.5)))

  encoding = feature_extractor(image, return_tensors="pt")
  # encoding.keys()
  model = TableTransformerForObjectDetection.from_pretrained("microsoft/table-transformer-structure-recognition")

  with torch.no_grad():
    outputs = model(**encoding)

    target_sizes = [image.size[::-1]]
    results = feature_extractor.post_process_object_detection(outputs, threshold=0.8, target_sizes=target_sizes)[0]

    return results, model



In [126]:

path_to_pdfs = "/Users/danielbasman/Desktop/Desktop - Daniel’s MacBook Pro/MyStuff/Classes/Human In Loop/folder_of_10K/"
path_to_csv = "/Users/danielbasman/Desktop/Desktop - Daniel’s MacBook Pro/MyStuff/Classes/Human In Loop/data_as_csv/"

data = get_image(path_to_pdfs)
print(data)

names = list(data)
model = TableTransformerForObjectDetection.from_pretrained("microsoft/table-transformer-detection")

for name in names[0:1]:
    item = data[name]
    img = item["images"]
    width = item["widths"]
    height = item["heights"]
    count = 0
    for i, j in enumerate(img):
        # print(i)
        results, feature_extractor = extract_features(j, width[i], height[i], model)
        final, max_score_box = sort_results(results)
        sorted_dict = convert_results(results)

        # print(final)
        # plot_results(img, results['scores'], results['labels'], results['boxes'])
        # plot_box(img, final)

        cropped_image = get_table_image(j, final)

        # results, model = table_structure_results(cropped_image, feature_extractor)
        # plot_results(cropped_image, results['scores'], results['labels'], results['boxes'])

        pdf_path = path_to_pdfs + name

        finals = []
        for x in final:
            finals.append((int(x)))

        reorder = [str(finals[1]), str(800 - finals[0]), str(finals[3]), str(800 - finals[2])]
        fina = ",".join(reorder)
        inpu = ([str(fina)])

        if int(reorder[1])-int(reorder[3]) < 700:
            print("TABLE PRESENT")
            count += 1
            # print()

        try:
            tables = camelot.read_pdf(pdf_path, pages=str(i), flavor='stream', table_areas = inpu)

        except:
            print("SUGGESTED TABLE WAS BLANK SPACE")
            continue

        print(tables[0].parsing_report)
        file_name = name.split(".")[0] + "/" + "page" + str(i) + ".csv"
        # print(file_name)
        # print("COLUMN COUNT: " + str(len(tables[0].df.columns)))
        print()
        if not os.path.exists(path_to_csv + name.split(".")[0] + "/"):
        # Create the folder if it does not exist
            os.makedirs(path_to_csv + name.split(".")[0] + "/")
            print(f"Folder created: " + str(path_to_csv + name.split(".")[0] + "/"))
        else:
        # If the folder already exists
            print(f"Folder already exists: " + str(path_to_csv + name.split(".")[0] + "/"))

        tables[0].to_csv(path_to_csv + file_name)

    print("COUNT: " + str(count))


10-K-EX2.pdf
10-K.pdf
{'10-K-EX2.pdf': {'name': '10-K-EX2.pdf', 'images': [<PIL.Image.Image image mode=RGB size=612x792 at 0x16A7CA230>, <PIL.Image.Image image mode=RGB size=612x792 at 0x16A7C9E10>, <PIL.Image.Image image mode=RGB size=612x792 at 0x16A7CA620>, <PIL.Image.Image image mode=RGB size=612x792 at 0x16BED0940>, <PIL.Image.Image image mode=RGB size=612x792 at 0x16A7CA140>, <PIL.Image.Image image mode=RGB size=612x792 at 0x16A7CA7A0>, <PIL.Image.Image image mode=RGB size=612x792 at 0x16BED0A90>, <PIL.Image.Image image mode=RGB size=612x792 at 0x16BED0AF0>, <PIL.Image.Image image mode=RGB size=612x792 at 0x16BED0730>, <PIL.Image.Image image mode=RGB size=612x792 at 0x16BED07C0>, <PIL.Image.Image image mode=RGB size=612x792 at 0x16BED0790>, <PIL.Image.Image image mode=RGB size=612x792 at 0x16BED38B0>, <PIL.Image.Image image mode=RGB size=612x792 at 0x16BED0CD0>, <PIL.Image.Image image mode=RGB size=612x792 at 0x16BED0B80>, <PIL.Image.Image image mode=RGB size=612x792 at 0x16BED0A

Some weights of the model checkpoint at microsoft/table-transformer-detection were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/models/detr/feature_

16 46 590 509
TABLE PRESENT
{'accuracy': 96.77, 'whitespace': 68.06, 'order': 1, 'page': 0}

Folder already exists: /Users/danielbasman/Desktop/Desktop - Daniel’s MacBook Pro/MyStuff/Classes/Human In Loop/data_as_csv/10-K-EX2/


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/models/detr/feature_extraction_detr.py:38: FutureWarning: The class DetrFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use DetrImageProcessor instead.
  warnings.warn(


38 47 569 380
TABLE PRESENT
{'accuracy': 80.31, 'whitespace': 70.27, 'order': 1, 'page': 1}

Folder already exists: /Users/danielbasman/Desktop/Desktop - Daniel’s MacBook Pro/MyStuff/Classes/Human In Loop/data_as_csv/10-K-EX2/


...

{'accuracy': 100.0, 'whitespace': 0.0, 'order': 1, 'page': 143}

Folder already exists: /Users/danielbasman/Desktop/Desktop - Daniel’s MacBook Pro/MyStuff/Classes/Human In Loop/data_as_csv/10-K-EX2/
21 34 561 261
TABLE PRESENT


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/camelot/parsers/stream.py:365: UserWarning: No tables found in table area 1
  warnings.warn(f"No tables found in table area {table_idx + 1}")
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/models/detr/feature_extraction_detr.py:38: FutureWarning: The class DetrFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use DetrImageProcessor instead.
  warnings.warn(


{'accuracy': 100.0, 'whitespace': 0.0, 'order': 1, 'page': 144}

Folder already exists: /Users/danielbasman/Desktop/Desktop - Daniel’s MacBook Pro/MyStuff/Classes/Human In Loop/data_as_csv/10-K-EX2/
5 91 585 575
TABLE PRESENT


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/camelot/parsers/stream.py:365: UserWarning: No tables found in table area 1
  warnings.warn(f"No tables found in table area {table_idx + 1}")
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/models/detr/feature_extraction_detr.py:38: FutureWarning: The class DetrFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use DetrImageProcessor instead.
  warnings.warn(


{'accuracy': 100.0, 'whitespace': 0.0, 'order': 1, 'page': 145}

Folder already exists: /Users/danielbasman/Desktop/Desktop - Daniel’s MacBook Pro/MyStuff/Classes/Human In Loop/data_as_csv/10-K-EX2/
8 53 583 428
TABLE PRESENT
{'accuracy': 99.08, 'whitespace': 55.3, 'order': 1, 'page': 146}

Folder already exists: /Users/danielbasman/Desktop/Desktop - Daniel’s MacBook Pro/MyStuff/Classes/Human In Loop/data_as_csv/10-K-EX2/
COUNT: 110
